In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Flatten, Dense, Input, 
                                     Dropout, BatchNormalization,
                                     Conv2D, MaxPooling2D, 
                                     GlobalMaxPooling2D, GlobalAveragePooling2D,)

from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

In [2]:
def cv_show(img, name='image'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

In [4]:
def padding(img):
    out_w = 128
    out_h = 128
    top_size = int((128 - img.shape[0]) / 2)
    bottom_size = int((128 - img.shape[0]) / 2)
    dif_1 = 128 - (top_size + bottom_size + img.shape[0])
    top_size += dif_1
    left_size = int((128 - img.shape[1]) / 2)
    right_size = int((128 - img.shape[1]) / 2)
    dif_2 = 128 - (left_size + right_size + img.shape[1])
    left_size += dif_2    
    if img.shape[1] <= 128:
        constant = cv2.copyMakeBorder(img, 
                                  top_size, bottom_size, left_size, right_size, 
                                  borderType=cv2.BORDER_CONSTANT, 
                                  value=(255, 255, 255))
    else:
        constant = cv2.copyMakeBorder(img, 
                          top_size, bottom_size, 0, 0, 
                          borderType=cv2.BORDER_CONSTANT, 
                          value=(255, 255, 255))
        constant = cv2.resize(constant, (128, 128))
        
 
    return constant

In [26]:
def preprocessing_img(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 110, 255, cv2.THRESH_OTSU)   
    gaussian = cv2.GaussianBlur(thresh1, (3, 3), 1)
    kernel = np.zeros((3, 3), np.uint8)
    erosion_1 = cv2.erode(gaussian, kernel, iterations=1)

    img2 = 255 - erosion_1
    img2 = cv2.resize(img2, (64, 64))
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    
    
    return img2

In [27]:
def load_data(path):
    
    fold_list = os.listdir(train_dir)
    trainX_list = []
    trainY_list = []
    testX_list = []
    testY_list = []

    for f in fold_list:
        label = int(f)
        img_list = os.listdir(f'{train_dir}{f}')
        tmp_datasetX = []
        tmp_datasetY = []
        for img_p in img_list:
            try:
                img = cv2.imread(f'{train_dir}{f}/{img_p}')
#                 if (img.shape[0] != 128) or (img.shape[1] != 128):
#                     img = padding(img)
                img = preprocessing_img(img)
#                 print(img.shape)
                label = int(f)
                tmp_datasetX.append(img)
                tmp_datasetY.append(label)
            except Exception as e:
                print(f, img_p)
                print(e)
        
        trainX, testX, trainY, testY = train_test_split(tmp_datasetX, tmp_datasetY, test_size=0.2, random_state=42)
        trainX_list.extend(trainX)
        trainY_list.extend(trainY)
        testX_list.extend(testX)
        testY_list.extend(testY)
    trainX_ary = np.array(trainX_list)
    trainY_ary = np.array(trainY_list)
    testX_ary = np.array(testX_list)
    testY_ary = np.array(testY_list)
    print(f'trainX: {trainX_ary.shape}')
    print(f'trainY: {trainY_ary.shape}')
    print(f'testX: {testX_ary.shape}')
    print(f'testY: {testY_ary.shape}')
        
    return trainX_ary, testX_ary, trainY_ary, testY_ary

In [28]:
train_dir = r'./data-clean/'
df = pd.read_csv('word_idx_df.csv', converters={'idx':str})

In [29]:
trainX, testX, trainY, testY = load_data(train_dir)


trainX: (47930, 64, 64, 3)
trainY: (47930,)
testX: (12391, 64, 64, 3)
testY: (12391,)


In [30]:
trainY = keras.utils.to_categorical(trainY, num_classes=800)
testY = keras.utils.to_categorical(testY, num_classes=800)

In [31]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(47930, 64, 64, 3)
(12391, 64, 64, 3)
(47930, 800)
(12391, 800)


In [12]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(47930, 128, 128, 3)
(12391, 128, 128, 3)
(47930, 800)
(12391, 800)


In [32]:
import gc
gc.collect()

8463

In [66]:
input_tensor = Input(shape=(64, 64, 3))
model = keras.applications.Xception(include_top=False, 
                                    weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None,
                                    classes=800)
x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
# x = Dense(1024, activation='relu')(x)
# x = BatchNormalization()(x)
predictions = Dense(800, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

In [67]:
len(model.layers)

136

In [68]:
for layer in model.layers[:66]:
    layer.trainable = False
for layer in model.layers[66:]:
    layer.trainable = True

In [69]:
model.layers[66].name

'block8_sepconv1_act'

In [71]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers, callbacks

In [72]:
opt = optimizers.Adam(lr=0.001) 
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
    ]

In [73]:
model.fit(trainX, trainY, batch_size=32, epochs=50, verbose=2,
          validation_data=(testX, testY), callbacks=callbacks ) 

Epoch 1/50
1498/1498 - 63s - loss: 6.2648 - accuracy: 0.0171 - val_loss: 5.4186 - val_accuracy: 0.0392 - lr: 0.0010
Epoch 2/50
1498/1498 - 63s - loss: 4.2624 - accuracy: 0.1235 - val_loss: 3.8140 - val_accuracy: 0.1763 - lr: 0.0010
Epoch 3/50
1498/1498 - 64s - loss: 3.0933 - accuracy: 0.2834 - val_loss: 3.0437 - val_accuracy: 0.3036 - lr: 0.0010
Epoch 4/50
1498/1498 - 64s - loss: 2.3669 - accuracy: 0.4144 - val_loss: 2.5177 - val_accuracy: 0.3980 - lr: 0.0010
Epoch 5/50
1498/1498 - 64s - loss: 1.8513 - accuracy: 0.5215 - val_loss: 2.3458 - val_accuracy: 0.4513 - lr: 0.0010
Epoch 6/50
1498/1498 - 64s - loss: 1.4758 - accuracy: 0.6051 - val_loss: 2.2139 - val_accuracy: 0.4860 - lr: 0.0010
Epoch 7/50
1498/1498 - 64s - loss: 1.1560 - accuracy: 0.6771 - val_loss: 2.2985 - val_accuracy: 0.4837 - lr: 0.0010
Epoch 8/50
1498/1498 - 64s - loss: 0.8948 - accuracy: 0.7436 - val_loss: 2.1825 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 9/50
1498/1498 - 64s - loss: 0.7004 - accuracy: 0.7921 - val_loss: